In [6]:
import pandas as pd

classification1 = pd.read_excel('../data/train_Ametist/classification.xlsx', sheet_name='Материалы, изд, констр и оборуд')
classification2 = pd.read_excel('../data/train_Ametist/classification.xlsx', sheet_name='Машины и механизмы')
classification = pd.concat([classification1, classification2], axis=0)
train_data = pd.read_excel('../data/train_Ametist/train.xlsx')

clear_mask = classification.dropna(axis=0, how='any').iloc[:, 0].apply(lambda x: all(c in '0123456789.- ' for c in x))
cleared_classification = classification.dropna(axis=0, how='any')[clear_mask].rename(columns={'Классификатор строительных ресурсов': 'code', 'Unnamed: 1': 'name', 'Unnamed: 2': 'measure'})
cleared_classification = cleared_classification.reset_index(drop=True)
print(classification.shape[0], cleared_classification.shape[0])
cleared_classification.head()

216113 213083


,code,name,measure
0,01.1.01.01-0002,Детали фасонные коньковые к листам хризотилцем...,100 компл
1,23.65.12.190.01.1.01.01-0002-000,Детали фасонные коньковые к листам хризотилцем...,100 компл
2,01.1.01.02-0011,"Доска электротехническая дугостойкая (АЦЭИД), ...",т
3,23.65.12.190.01.1.01.02-0011-000,"Доска электротехническая дугостойкая (АЦЭИД), ...",т
4,01.1.01.04-1018,"Листы хризотилцементные волнистые, профиль 40/...",м2


## POS and DEP SpaCy

### Feature stripping

In [339]:
import spacy
import string
from spacy import displacy


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))


nlp = spacy.load('ru_core_news_sm')  # нужно скачать: python -m spacy download ru_core_news_sm

In [323]:
def strip_features_and_normaize(text):
    text = re.sub(r'\s*-\s*', '/', text)
    text = re.sub(r'\(.*?\)|\[.*?\]|\{.*?\}', '', text)
    result = nlp(text)
    filtered_result = filter(lambda x: x.pos_ in ['NOUN', 'ADJ', 'VERB', 'PROPN'] and 
                             x.dep_ in ['ROOT', 'amod', 'nsubj', 
                                        'advmod', 'nmod', 'nsubjpass', 
                                        'nmod:npmod', 'nmod:poss',
                                        'nmod:tmod', 'obl'] and
                             len(x.text) > 2, result)

    return ' '.join(list([e.lemma_.strip() for e in filtered_result]))


for idx in range(0, 200, 20):
    sample = cleared_classification.name.iloc[idx]
    clean_sample = strip_features_and_normaize(sample)
    print(idx, sample, '--->', clean_sample)

0 Детали фасонные коньковые к листам хризотилцементным волнистым ---> деталь фасонный коньковый лист хризотилцементным
20 Листы хризотилцементные волнистые, профиль 40/150, толщина 5,8 мм ---> лист хризотилцементные волнистый
40 Листы хризотилцементные плоские прессованные, толщина 20 мм ---> лист хризотилцементные плоский прессованные
60 Шнур асбестовый общего назначения ШАОН, диаметр 18-25 мм ---> шнур общий назначение шаон
80 Бумага асбестовая электроизоляционная БЭ, толщина 0,2-0,3 мм ---> бумага асбестовый электроизоляционный
100 Добавка (фибра) из полипропиленового волокна армирующая для бетонов ---> добавка полипропиленовый волокно армировать бетон
120 Прокладки из паронита ПМБ, толщина 4 мм, диаметр 50 мм ---> прокладка паронит пмб
140 Прокладки паронитовые для опорного кронштейна, размеры 138х40х2 мм ---> прокладка паронитовые опорный кронштейн
160 Прокладки из паронита ПМБ, толщина 2 мм, диаметр 100 мм ---> прокладка паронит пмб
180 Прокладки из паронита ПМБ, толщина 3 мм, ди

In [324]:
for idx in range(0, 200, 20):
    sample = train_data.record_name.iloc[idx]
    clean_sample = strip_features_and_normaize(sample)
    print(idx, sample, '--->', clean_sample)

0 DIN -рейка оцинкованная ТН35-7.5 100 см (Chint) ---> din оцинкованная тн35/7.5
20 Арматура композитная АСК-6 Н бухта 50 м(срок постаки 1-2 раб дня) ---> арматура композитный бухта
40 Балка монолитная БМ2 (6000x500x400) В25 ---> балка монолитный
60 (ДПМ-1 EI-60)  дверь металлическая противопожарная Комплект фурнитуры по ТЗ Подготовка под ЭМЗ ---> дверь металлический противопожарный фурнитура эмз
80 Дверь влагостойкая однопольная, правая 1050x2100 толщина стены 250 мм ---> дверь влагостойкий однопольная стена
100 Болт анкерный М 16x180 оцинк. ---> болт анкерный оцинк
120 Наконечник кабельный кольцевой 4кв.мм под болт М8 2СТЕ8М DKC ---> наконечник кабельный кольцевой болт
140 Болт М10х 40 DIN933 класс прочности 8.8 оц. Срок поставки 2-3 раб дня ---> болт м10х класс прочность срок поставка раб день
160 Болт с полной резьбой оц. М8х30 ---> болт полный резьба
180 Метизный двор Болт с шестигранной головкой 10x40 DIN 933, 100 шт. 4607159065209 ---> метизный двор болт шестигранной головка


### Feature extraction

In [335]:
import re


def extract_features(text):
    text = re.sub(r'\s*[-\*]\s*', '//', text)
    text = re.sub(r'\(.*?\)|\[.*?\]|\{.*?\}', '', text)
    result = nlp(text)
    values = filter(lambda x: (x.pos_ in {'ADJ', 'NUM', 'PROPN'} or x.dep_ in {'appos', 'nummod'}), result)
    values = set(map(lambda x: x.lemma_.strip(), values))
    values = {t.replace('//', '-') for t in values}
    features_dict = dict()
    for token in result:
        if token.pos_ not in ['NOUN'] or len(token.text) < 3:
            continue
        token_children = set(map(lambda x: x.lemma_.strip(), token.children))
        token_values = token_children & values
        values -= token_values
        if len(token_values) == 0:
            continue
        features_dict[token.lemma_.strip()] = ' '.join(token_values)

    return features_dict, values


for idx in range(0, 300, 20):
    sample = cleared_classification.name.iloc[idx]
    features = extract_features(sample)
    print(idx, sample, '--->', features)

0 Детали фасонные коньковые к листам хризотилцементным волнистым ---> ({'деталь': 'коньковый хризотилцементным волнистый'}, {'фасонный'})
20 Листы хризотилцементные волнистые, профиль 40/150, толщина 5,8 мм ---> ({'профиль': '40/150', 'толщина': '5,8'}, {'волнистый', 'хризотилцементные'})
40 Листы хризотилцементные плоские прессованные, толщина 20 мм ---> ({'толщина': '20'}, {'плоский', 'хризотилцементные', 'прессованные'})
60 Шнур асбестовый общего назначения ШАОН, диаметр 18-25 мм ---> ({'шнур': 'асбестовый', 'назначение': 'общий шаон'}, {'18-25'})
80 Бумага асбестовая электроизоляционная БЭ, толщина 0,2-0,3 мм ---> ({'бумага': 'бэ'}, {'асбестовый', '0,2-0,3', 'электроизоляционный'})
100 Добавка (фибра) из полипропиленового волокна армирующая для бетонов ---> ({'волокно': 'полипропиленовый'}, set())
120 Прокладки из паронита ПМБ, толщина 4 мм, диаметр 50 мм ---> ({'прокладка': 'толщина', 'паронит': 'пмб', 'диаметр': '50'}, {'4'})
140 Прокладки паронитовые для опорного кронштейна, раз

In [334]:
for idx in range(0, 400, 20):
    sample = train_data.record_name.iloc[idx]
    features = extract_features(sample)
    print(idx, sample, '--->', features)

0 DIN -рейка оцинкованная ТН35-7.5 100 см (Chint) ---> ({'тн35//7.5': '100'}, {'din-рейка'})
20 Арматура композитная АСК-6 Н бухта 50 м(срок постаки 1-2 раб дня) ---> ({'аск//6': 'композитный н', 'бухта': '50'}, {'аск-6'})
40 Балка монолитная БМ2 (6000x500x400) В25 ---> ({'балка': 'бм2 в25'}, {'монолитный'})
60 (ДПМ-1 EI-60)  дверь металлическая противопожарная Комплект фурнитуры по ТЗ Подготовка под ЭМЗ ---> ({'дверь': 'комплект', 'фурнитура': 'тз'}, {'подготовка', 'эмз', 'металлический', 'противопожарный'})
80 Дверь влагостойкая однопольная, правая 1050x2100 толщина стены 250 мм ---> ({'толщина': '1050x2100', 'стена': '250'}, {'однопольная', 'правый', 'влагостойкий'})
100 Болт анкерный М 16x180 оцинк. ---> ({'оцинк': 'болт'}, {'16x180', 'анкерный', 'м'})
120 Наконечник кабельный кольцевой 4кв.мм под болт М8 2СТЕ8М DKC ---> ({'наконечник': '4кв.мм', 'болт': 'м8'}, {'кабельный'})
140 Болт М10х 40 DIN933 класс прочности 8.8 оц. Срок поставки 2-3 раб дня ---> ({'болт': 'din933', 'м10х': 

In [336]:
re.sub(r'\s*[-\*]\s*', '//', 'бур SDS+ 18*260')

'бур SDS+ 18//260'